<h2>Titanic project</h2>

<h4>Import essential library</h4>

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
os.path.abspath("")

'C:\\Users\\CutieUnicorn\\Downloads'

<h4>Import data</h4>

In [3]:
# Raw URLs to CSV files
test_data_url = 'https://raw.githubusercontent.com/Yosapatjj/Titanic-ML/fc58eb1c716b1e36399afb3cc16b7b49d4c3416f/test.csv'
train_data_url = 'https://raw.githubusercontent.com/Yosapatjj/Titanic-ML/fc58eb1c716b1e36399afb3cc16b7b49d4c3416f/train.csv'

# Read CSV files into DataFrames
test_data = pd.read_csv(test_data_url)
train_data = pd.read_csv(train_data_url)

In [4]:
test_data.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


<h4>EDA and Data manipulation</h4>

In [5]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
age_median = train_data['Age'].median()
train_data['Age'].fillna(age_median, inplace = True)

In [8]:
bins = [0, 5, 20, 30, 60, float('inf')]
labels = ['Kid', 'Teenager', 'Young Adult', 'Adult', 'Elderly']
train_data['Age_cat'] = pd.cut(train_data['Age'], bins=bins, labels=labels, right=False)

In [9]:
selected_column = ['Survived','Pclass','Sex','SibSp','Parch','Fare','Age_cat']
clean_train = train_data[selected_column].copy()
clean_train.head(10)

,Survived,Pclass,Sex,SibSp,Parch,Fare,Age_cat
0,0,3,male,1,0,7.2500,Young Adult
1,1,1,female,1,0,71.2833,Adult
2,1,3,female,0,0,7.9250,Young Adult
3,1,1,female,1,0,53.1000,Adult
4,0,3,male,0,0,8.0500,Adult
5,0,3,male,0,0,8.4583,Young Adult
6,0,1,male,0,0,51.8625,Adult
7,0,3,male,3,1,21.0750,Kid
8,1,3,female,0,2,11.1333,Young Adult
9,1,2,female,1,0,30.0708,Teenager


In [10]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
age_median = test_data['Age'].median()
test_data['Age'].fillna(age_median, inplace = True)

In [12]:
bins = [0, 5, 20, 30, 60, float('inf')]
labels = ['Kid', 'Teenager', 'Young Adult', 'Adult', 'Elderly']
test_data['Age_cat'] = pd.cut(test_data['Age'], bins=bins, labels=labels, right=False)

selected_column = ['Pclass','Sex','SibSp','Parch','Fare','Age_cat']
clean_test = test_data[selected_column].copy()
clean_test.head(10)

,Pclass,Sex,SibSp,Parch,Fare,Age_cat
0,3,male,0,0,7.8292,Adult
1,3,female,1,0,7.0000,Adult
2,2,male,0,0,9.6875,Elderly
3,3,male,0,0,8.6625,Young Adult
4,3,female,1,1,12.2875,Young Adult
5,3,male,0,0,9.2250,Teenager
6,3,female,0,0,7.6292,Adult
7,2,male,1,1,29.0000,Young Adult
8,3,female,0,0,7.2292,Teenager
9,3,male,2,0,24.1500,Young Adult


In [14]:
# Define the categorical data in the data
categorical_columns = ['Pclass', 'Sex', 'Age_cat']

# Perform one-hot encoding for categorical variables
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(clean_train[categorical_columns])

# Split data into features and target variable
x = encoded_data
y = clean_train['Survived']

# Split data into train, validation, and test sets
# Train data is 80% of dataset || Test data is at 20%
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=77)

# Split 0.25 of Test data will be use as validation dataset
x_val,   x_test, y_val,   y_test = train_test_split(x_temp, y_temp, test_size=0.25, random_state=77)

<h4> One Hot Encoder </h4>

In [15]:
# Initialize and train the model
model = RandomForestClassifier()
model.fit(x_train, y_train)

# Make predictions
predictions = model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8


In [16]:
encoded_test = encoder.fit_transform(clean_test[categorical_columns])
X_test = encoded_test
y_pred = model.predict(X_test)
result_df = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_pred})
result_df.head(5)
result_df.to_csv('01_OneHot.csv',index = False)

<h4> One Hot Encoder with validation</h4>

In [17]:
# Initialize and train the model
model_OneHot_val = RandomForestClassifier()
model_OneHot_val.fit(x_train, y_train)

# Make predictions on the validation set
val_predictions = model_OneHot_val.predict(x_val)

# Evaluate the model on the validation set
val_accuracy = accuracy_score(y_val, val_predictions)
print("Validation Accuracy:", val_accuracy)

# Make predictions on the test set
test_predictions = model_OneHot_val.predict(x_test)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print("Test Accuracy:", test_accuracy)

Validation Accuracy: 0.7985074626865671
Test Accuracy: 0.8


In [18]:
encoded_test = encoder.fit_transform(clean_test[categorical_columns])
X_test = encoded_test
y_pred = model_OneHot_val.predict(X_test)
result_OneHot = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_pred})
result_OneHot.head(5)
result_OneHot.to_csv('02_OneHot_Validate.csv',index = False)

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the parameter distributions
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}

# Initialize the randomized search with the RandomForestClassifier model and parameter distributions
random_search = RandomizedSearchCV(RandomForestClassifier(), param_dist, n_iter=100, cv=5, random_state=77)

# Perform randomized search on the training data (including validation data)
random_search.fit(x_train, y_train)

# Get the best parameters found by randomized search
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Get the best model
best_model = random_search.best_estimator_

# Make predictions on the test set using the best model
test_predictions = best_model.predict(x_test)

# Evaluate the best model on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print("Test Accuracy:", test_accuracy)


Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 15, 'n_estimators': 261}
Test Accuracy: 0.7555555555555555


In [ ]:
X_test = encoded_test
y_pred = best_model.predict(X_test)
result_RandSearch = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_pred})
result_RandSearch.head(5)
result_RandSearch.to_csv('02_result_RandSearch.csv',index = False)

In [30]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint

# Define the hyperparameter distributions
param_dist = {
    'n_estimators': list(range(100, 501)), 
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': list(range(10, 21)),  
    'min_samples_leaf': list(range(1, 16))  
}

# Create a CatBoostClassifier object
catboost_model = CatBoostClassifier(loss_function='Logloss', verbose=False)

# Perform grid search with cross-validation
grid_search = GridSearchCV(catboost_model, param_dist, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# Get the best parameters
catboost_params = grid_search.best_params_
print("Best Parameters:", catboost_params)

# Train the model with the best parameters
best_catboost = CatBoostClassifier(**catboost_params)
best_catboost.fit(x_train, y_train)

# Make predictions on the test set
test_predictions = best_catboost.predict(x_test)

# Evaluate the best model on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print("Test Accuracy:", test_accuracy)

C:\Users\CutieUnicorn\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1984950 fits failed out of a total of 1984950.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
132330 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\CutieUnicorn\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\CutieUnicorn\anaconda3\lib\site-packages\catboost\core.py", line 5128, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "C:\Users\CutieUnicorn\anaconda3\lib\sit

TypeError: __init__() got an unexpected keyword argument 'min_samples_leaf'